# Trainer
千帆Python SDK 在使用[resource API实现发起训练微调](./console-finetune.ipynb)之外，还提供了Trainer API，可以更方便地实现一体化的训练微调pipeline。同时提供了状态事件回调函数的注册，通过事件分发实现训练流程状态事件的监控。


本例将基于qianfan==0.2.2展示通过Dataset加载本地数据集，并上传到千帆平台，基于ERNIE-Bot-turbo进行fine-tune，并使用Model进行批量跑评估数据，直到最终完成服务发布，并最终实现服务调用的完整过程。

In [ ]:
! pip install "qianfan>=0.2.2" -U

In [1]:
import qianfan
qianfan.__version__

'0.2.2'

## 前置准备
- 初始化千帆安全认证AK、SK

In [2]:
import os 

os.environ["QIANFAN_ACCESS_KEY"] = "your_ak"
os.environ["QIANFAN_SECRET_KEY"] = "your_sk"

## 数据集加载

千帆SDK提供了数据集实现帮助我们可以快速的加载本地的数据集到内存，并通过设定DataSource数据源以保存至本地和千帆平台。

In [3]:
from qianfan.dataset import Dataset

# 加载本地数据集
ds: Dataset = Dataset.load(data_file="./data/fin_cqa_train.jsonl")
ds.list()

[[{'prompt': '下文中市场价格下降导致市场价格下降事件对应的结果涉及的产品是？在国际奶粉价格下降压力下,国内奶价仍有下降空间',
   'response': [['奶']]}],
 [{'prompt': '下文中市场价格下降导致市场价格下降事件对应的结果涉及的地区是？在国际奶粉价格下降压力下,国内奶价仍有下降空间',
   'response': [['国内']]}],
 [{'prompt': '下文中有哪些因果事件？随着供应缩减，需求回升，高碳铬铁价格企稳回升，预期短期内或仍维持偏强态势，但高碳铬铁产能依然偏向过剩，后期价格弹升空间有限',
   'response': [['需求增加导致市场价格提升']]}],
 [{'prompt': '下文中需求增加导致市场价格提升事件对应的原因涉及的产品是？随着供应缩减，需求回升，高碳铬铁价格企稳回升，预期短期内或仍维持偏强态势，但高碳铬铁产能依然偏向过剩，后期价格弹升空间有限',
   'response': [['高碳铬铁']]}],
 [{'prompt': '下文中需求增加导致市场价格提升事件对应的原因涉及的行业是？随着供应缩减，需求回升，高碳铬铁价格企稳回升，预期短期内或仍维持偏强态势，但高碳铬铁产能依然偏向过剩，后期价格弹升空间有限',
   'response': [['我们无法得知，可能需要更多内容说明。']]}],
 [{'prompt': '下文中需求增加导致市场价格提升事件对应的结果涉及的产品是？随着供应缩减，需求回升，高碳铬铁价格企稳回升，预期短期内或仍维持偏强态势，但高碳铬铁产能依然偏向过剩，后期价格弹升空间有限',
   'response': [['高碳铬铁']]}],
 [{'prompt': '下文中有哪些因果事件？但由于7-ACA市场价格大幅下跌，跌幅超过50%，导致公司相关原料药销售价格随之下跌',
   'response': [['市场价格下降导致市场价格下降']]}],
 [{'prompt': '下文中市场价格下降导致市场价格下降事件对应的原因涉及的产品是？但由于7-ACA市场价格大幅下跌，跌幅超过50%，导致公司相关原料药销售价格随之下跌',
   'response': [['7-ACA']]}],
 [{'prompt': '下文中市场价格下降导致市场价

从本地数据集上传到BOS

In [4]:
# 保存到千帆平台
from qianfan.dataset import QianfanDataSource
from qianfan.resources.console import consts as console_consts

bos_bucket_name = "<bos_bucket>"
bos_bucket_file_path = "/bos_path/"


# 创建千帆数据集，并上传保存
qianfan_data_source = QianfanDataSource.create_bare_dataset(
    name="sdk_trainer_ds_022h",
    template_type=console_consts.DataTemplateType.NonSortedConversation,
    storage_type=console_consts.DataStorageType.PrivateBos,
    storage_id=bos_bucket_name,
    storage_path=bos_bucket_file_path,
)

ds.save(qianfan_data_source, replace_source=True)

True

In [5]:
from qianfan.trainer.consts import ActionState, ServiceType
from qianfan.resources.console import consts as console_consts
from qianfan.trainer.configs import TrainConfig, DeployConfig
from qianfan.resources import QfMessages
from qianfan.trainer import LLMFinetune
from typing import cast
from qianfan.utils import enable_log
import logging

enable_log(logging.DEBUG)

### LLMFinetune 训练
`LLMFinetune` 实现了SFT逻辑的trainer，它内部组装了SFT所需要的基本`Pipeline`, 用于串联数据->训练->模型发布->服务调用等步骤

In [6]:
from qianfan.trainer.consts import PeftType

trainer = LLMFinetune(
    train_type="ERNIE-Bot-turbo-0725",
    train_config=TrainConfig(
        epoch=1,
        learning_rate=0.0003,
        max_seq_len=4096,
        peft_type=PeftType.LoRA,
    ),
    dataset=ds,
)

### 运行任务
同步运行trainer，训练直到模型发布完成

In [7]:
trainer.run()

[DEBUG] [12-08 11:40:00] base.py:222 [t:140481718310720]: action[Pipeline][OVQctg0L23] Preceding
[DEBUG] [12-08 11:40:00] base.py:222 [t:140481718310720]: action[LoadDataSetAction][JkGPa954jn] Preceding
[DEBUG] [12-08 11:40:00] actions.py:85 [t:140481718310720]: [load_dataset_action] prepare train-set
[INFO] [12-08 11:40:06] data_source.py:1044 [t:140481718310720]: data releasing, keep rolling
[INFO] [12-08 11:40:12] data_source.py:1044 [t:140481718310720]: data releasing, keep rolling
[INFO] [12-08 11:40:15] data_source.py:1044 [t:140481718310720]: data releasing, keep rolling
[INFO] [12-08 11:40:18] data_source.py:1044 [t:140481718310720]: data releasing, keep rolling
[INFO] [12-08 11:40:22] data_source.py:1053 [t:140481718310720]: data releasing succeeded
[DEBUG] [12-08 11:40:22] actions.py:91 [t:140481718310720]: [load_dataset_action] dataset loaded successfully
[DEBUG] [12-08 11:40:22] base.py:226 [t:140481718310720]: action[LoadDataSetAction][JkGPa954jn] Done
[DEBUG] [12-08 11:40

获取finetune任务输出：

In [8]:
trainer.output

{'task_id': 17320,
 'job_id': 9098,
 'model_id': 10268,
 'model_version_id': 12722,
 'model': <qianfan.trainer.model.Model at 0x7fc40ac23ad0>}

### 运行批量评估推理
Model支持模型批量运行评估数据集，并保存到千帆平台

调用Fine-tune得到的Model对象的`batch_run_on_qianfan`发起批量任务，这可能会持续数十分钟

In [17]:
from qianfan.trainer import Model
from qianfan.dataset import Dataset

# 首先需要先加载测试数据集，这里以加载平台预置数据集为例子：
test_ds = Dataset.load(qianfan_dataset_id=15074, is_download_to_local=False)

# 从训练结果中获取模型对象
m: Model = trainer.output["model"]

# 运行批量任务获取结果数据集
result_ds: Dataset = m.batch_run_on_qianfan(test_ds)

[INFO] [12-08 12:22:22] model.py:295 [t:140481718310720]: start to create evaluation task in model
[DEBUG] [12-08 12:22:25] model.py:318 [t:140481718310720]: create evaluation task in model response: {'log_id': '3646090660', 'result': {'evalId': 2419}}
[INFO] [12-08 12:22:25] model.py:319 [t:140481718310720]: start to polling status of evaluation task 2419
[DEBUG] [12-08 12:22:26] model.py:325 [t:140481718310720]: current evaluation task info: QfResponse(code=200, headers={'Date': 'Fri, 08 Dec 2023 04:22:26 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '858', 'tracecode': '13466531631049976330120812', 'Set-Cookie': 'BAIDUID=395E0EBE86F8CDE862BD8BD0F5B7C8B6:FG=1; expires=Sat, 07-Dec-24 04:22:26 GMT; max-age=31536000; path=/; domain=.baidu.com; version=1', 'P3P': 'CP=" OTI DSP COR IVA OUR IND COM "', 'X-Bce-Request-Id': '43b62329-6ff5-465a-ae16-9614afbe3b9e', 'X-Bce-Gateway-Region': 'GZ'}, body={'log_id': '3859990056', 'result': {'evaluationId': 2419, 'name':

通过这种方式运行完成后，可以直接在本地拿到一份批量运行的结果，我们可以通过dataset.list查看其中的部分数据：

In [ ]:
result_ds.list([i for i in range(3,6)])

[[{'model_response': [{'content': '下文中需求增加导致市场价格提升事件对应的结果涉及的产品是高碳铬铁。',
     'model_version_id': 12722,
     'tag': 'm_17320_9098>V1'}],
   'prompt': '下文中需求增加导致市场价格提升事件对应的结果涉及的产品是？随着供应缩减，需求回升，高碳铬铁价格企稳回升，预期短期内或仍维持偏强态势，但高碳铬铁产能依然偏向过剩，后期价格弹升空间有限',
   'response': [['高碳铬铁']],
   'source_entity_id': '2d469d3fb6dea6e5ca6a1c7c0e91dceee6f918b495e22bedd1b496fc3f92147f'}],
 [{'model_response': [{'content': '根据上下文，需求增加导致市场价格上涨的事件涉及的产品是高碳铬铁。',
     'model_version_id': 12722,
     'tag': 'm_17320_9098>V1'}],
   'prompt': '下文中需求增加导致市场价格提升事件对应的原因涉及的产品是？随着供应缩减，需求回升，高碳铬铁价格企稳回升，预期短期内或仍维持偏强态势，但高碳铬铁产能依然偏向过剩，后期价格弹升空间有限',
   'response': [['高碳铬铁']],
   'source_entity_id': 'b7e4ea8632312142d853344214d34256d20a10d9fb890a81417a2ea9d2e70f04'}],
 [{'model_response': [{'content': '- 产品：煤炭\n- 原因：煤炭价格大幅上涨',
     'model_version_id': 12722,
     'tag': 'm_17320_9098>V1'}],
   'prompt': '下文中市场价格提升导致运营成本提升事件对应的原因涉及的产品是？利润和每股收益大幅下降的主要原因是煤炭价格大幅上涨，导致公司投资的火电企业的煤炭成本大幅增加###但报告期内受煤炭行业去产能改革的影响，公司煤炭前三季度平均采购价格同比上涨44.53%，导致成本涨幅抵消了销售价

在完成模型的批量运行后，我们可以对模型有一个简单的体感评估，如果效果不错，我们可以选择发布成服务以最终应用生产：

In [18]:
from qianfan.trainer.model import Model, Service, model_deploy
from qianfan.trainer.consts import ServiceType
from qianfan.resources.console.consts import DeployPoolType

sft_svc: Service = m.deploy(DeployConfig(
    name="sdkcqasvc",
    endpoint_prefix="sdkcqa1",
    replicas=1, # 副本数， 与qps强绑定
    pool_type=DeployPoolType.PrivateResource, # 私有资源池
    service_type=ServiceType.Chat,
))


[INFO] [12-08 12:40:10] model.py:497 [t:140481718310720]: ready to deploy service with model 10268/12722
[INFO] [12-08 12:40:18] model.py:529 [t:140481718310720]: please check web console `https://console.bce.baidu.com/qianfan/ais/console/onlineService`,for service  deployment payment.
[INFO] [12-08 12:40:52] model.py:529 [t:140481718310720]: please check web console `https://console.bce.baidu.com/qianfan/ais/console/onlineService`,for service  deployment payment.
[INFO] [12-08 12:41:22] model.py:529 [t:140481718310720]: please check web console `https://console.bce.baidu.com/qianfan/ais/console/onlineService`,for service  deployment payment.
[INFO] [12-08 12:41:53] model.py:529 [t:140481718310720]: please check web console `https://console.bce.baidu.com/qianfan/ais/console/onlineService`,for service  deployment payment.
[INFO] [12-08 12:42:24] model.py:529 [t:140481718310720]: please check web console `https://console.bce.baidu.com/qianfan/ais/console/onlineService`,for service  deplo

使用Finetune之后的模型服务和原始的预置模型服务调用：

In [26]:
from qianfan import ChatCompletion
### 使用Model & Service调用模型

problem="下文中有哪些因果事件？无取向硅钢广泛应用于铁芯等电机零部件其产量的持续提升导致市场竞争愈发激烈，价格进一步降低，从而有效减少新能源汽车驱动电机行业内企业的成本支出"

#获取服务对象，即ChatCompletion等类型的对象
chat_comp: ChatCompletion = sft_svc.get_res()
sft_chat_resp = chat_comp.do(messages=[{"content": problem, "role": "user"}])
sft_chat_resp["result"]

[INFO] [12-08 13:30:55] openapi_requestor.py:134 [t:140481718310720]: requesting llm api endpoint: /chat/ex1ndqpo_sdkcqa1


'因果事件如下：\n- 因果事件：无取向硅钢产量的持续提升导致市场竞争愈发激烈，价格进一步降低，从而有效减少新能源汽车驱动电机行业内企业的成本支出'

In [28]:
svc = ChatCompletion(model="ERNIE-Bot-turbo")

chat_resp = svc.do(messages=[{"content": problem, "role": "user"}])
chat_resp["result"]

[INFO] [12-08 13:31:55] openapi_requestor.py:134 [t:140481718310720]: requesting llm api endpoint: /chat/eb-instant


'因果事件如下：\n\n事件类型：应用\n- 事件触发词：应用于\n- 事件论元：\n  \t- 主体：无取向硅钢\n  \t- 客体：铁芯等电机零部件\n\n事件类型：提升\n- 事件触发词：提升\n- 事件论元：\n  \t- 主体：无取向硅钢的产量\n  \t- 结果：持续持续提升\n\n事件类型：降低\n- 事件触发词：降低\n- 事件论元：\n  \t- 主体：无取向硅钢的价格\n  \t- 原因：无取向硅钢的产量持续提升\n\n事件类型：减少\n- 事件触发词：减少\n- 事件论元：\n  \t- 主体：新能源汽车驱动电机行业内企业的成本支出\n  \t- 原因：无取向硅钢的价格进一步降低\n  \t- 影响：有效减少新能源汽车驱动电机行业内企业的成本支出\n  \t- 结论：与降低价格的因素有因果关系'

## EventHandler

如果需要在训练过程中监控每个阶段的各个节点的状态，可以通过事件回调函数来实现，通过事件的对应的action_state可以获取当前的action的运行情况以实现对应的业务回调，插入自定义逻辑

In [ ]:
from qianfan.trainer.event import Event, EventHandler

testset: Dataset = Dataset.load(data_file="./data/fin_cqa_test.jsonl")
# 定义自己的EventHandler，并实现dispatch方法
class InferAfterSFT(EventHandler):
    target_action: str
    def __init__(self, target_action: str) -> None:
        super().__init__()
        self.target_action = target_action

    def dispatch(self, event: Event) -> None:
        print("receive: <", event)
        if self.target_action == event.action_id and event.action_state == ActionState.Done:
            svc = cast(Service, event.data["service"])
            print("svc", svc)
            for row in testset.list():
                msgs = QfMessages()
                msgs.append(row[0][0]["prompt"], "user")
                svc.exec({"messages":"msgs"})
                print("row infer result", row)
            

eh = InferAfterSFT(target_action=trainer.ppls[0].id)
trainer.register_event_handler(eh)
trainer.run()

### 任务恢复

针对网络中断，服务不稳定等重试无法覆盖的场景，SDK提供了`resume()`以恢复训练过程，这里以LLMFinetune中断后恢复为例：

In [ ]:
trainer.run()

[DEBUG] [12-07 21:54:22] base.py:222 [t:139789057857344]: action[Pipeline][HFXZowejkt] Preceding
[DEBUG] [12-07 21:54:22] base.py:222 [t:139789057857344]: action[LoadDataSetAction][LWOkl2YCun] Preceding
[DEBUG] [12-07 21:54:22] actions.py:85 [t:139789057857344]: [load_dataset_action] prepare train-set
[INFO] [12-07 21:54:28] data_source.py:1044 [t:139789057857344]: data releasing, keep rolling
[INFO] [12-07 21:54:30] data_source.py:1044 [t:139789057857344]: data releasing, keep rolling
[INFO] [12-07 21:54:33] data_source.py:1044 [t:139789057857344]: data releasing, keep rolling
[INFO] [12-07 21:54:38] data_source.py:1044 [t:139789057857344]: data releasing, keep rolling
[INFO] [12-07 21:54:41] data_source.py:1053 [t:139789057857344]: data releasing succeeded
[DEBUG] [12-07 21:54:41] actions.py:91 [t:139789057857344]: [load_dataset_action] dataset loaded successfully
[DEBUG] [12-07 21:54:41] base.py:226 [t:139789057857344]: action[LoadDataSetAction][LWOkl2YCun] Done
[DEBUG] [12-07 21:54

APIError: api return error, code: 500002, msg: auth failed, no access

In [ ]:
trainer.resume()

[DEBUG] [12-07 22:00:58] base.py:222 [t:139789057857344]: action[Pipeline][HFXZowejkt] Preceding
[DEBUG] [12-07 22:00:58] base.py:222 [t:139789057857344]: action[TrainAction][F50ICFnguL] Preceding
[INFO] [12-07 22:00:58] actions.py:390 [t:139789057857344]: [train_action] resume from created job 17304/9077
[INFO] [12-07 22:00:58] actions.py:352 [t:139789057857344]: [train_action] fine-tune running... current status: RUNNING, check vdl report in https://console.bce.baidu.com/qianfan/visualdl/index?displayToken=eyJydW5JZCI6InJ1bi10MnlzaWQ3NjE1Z3N0Zm11In0=
[INFO] [12-07 22:01:29] actions.py:352 [t:139789057857344]: [train_action] fine-tune running... current status: RUNNING, check vdl report in https://console.bce.baidu.com/qianfan/visualdl/index?displayToken=eyJydW5JZCI6InJ1bi10MnlzaWQ3NjE1Z3N0Zm11In0=
[INFO] [12-07 22:02:00] actions.py:352 [t:139789057857344]: [train_action] fine-tune running... current status: RUNNING, check vdl report in https://console.bce.baidu.com/qianfan/visualdl/i

[INFO] [12-07 22:36:20] model.py:199 [t:139789057857344]: model publishing keep polling, current status FINISH
[INFO] [12-07 22:36:20] model.py:233 [t:139789057857344]: model ready to publish
[INFO] [12-07 22:36:21] model.py:239 [t:139789057857344]: check model publish status: Creating
[INFO] [12-07 22:36:51] model.py:239 [t:139789057857344]: check model publish status: Ready
[INFO] [12-07 22:36:51] model.py:241 [t:139789057857344]: model 10248/12701 published successfully
[DEBUG] [12-07 22:36:51] actions.py:471 [t:139789057857344]: [model publish] model: 17304_9077 has been published.
[DEBUG] [12-07 22:36:51] base.py:226 [t:139789057857344]: action[ModelPublishAction][YE5tSNDKlJ] Done
[DEBUG] [12-07 22:36:51] base.py:226 [t:139789057857344]: action[Pipeline][HFXZowejkt] Done
